# Import sxdm

In [1]:
%matplotlib qt
from sxdm import *

# Import MDA

In [2]:
github_clone = '/Users/will/Documents/sxdm/'
save_dir = '/Users/will/Desktop/sxdm_test'
save_file_name = 'file'
file = f'{save_dir}/{save_file_name}.h5'

In [5]:
import_mda(mda_path=f'{github_clone}tests/test_data/data/',
             hdf5_save_directory=save_dir,
             hdf5_save_filename=save_file_name)

1it [00:00,  6.80it/s]


# Import Images

In [6]:
import_images(
    file=file,
    images_loc=f'{github_clone}tests/test_data/images/',
    scans=False,
    fill_num=3,
    import_type='uint32',
    )

100%|██████████| 2/2 [00:05<00:00,  2.70s/it]


# Save MetaData

In [9]:
# Display a preset detector channel input that the User can copy
#disp_det_chan(file)

# Example Output

fluor = {
    'Fe1':33,
    'Fe2':3,
    'Fe3':4,
    'Fe4':30,
    }

roi = {
    'ROI_1':5,
    'ROI_2':6,
    'ROI_3':7,
    }

detector_scan = {
    'Main_Scan':65,
    }

filenumber = 10
sample_theta = 56
hybrid_x = 67
hybrid_y = 68

mis = {
    '2Theta':55,
    'Storage_Ring_Current':70,
    'Relative_r_To_Detector':54,
    }

xrf = {
    'Fe1':2,
    'Fe2':2,
    'Fe3':2,
    'Full':2
    }

setup_det_chan(file, fluor, roi,
               detector_scan, filenumber,
               sample_theta, hybrid_x, hybrid_y,
               mis, xrf=xrf)

# Creating Frameset

In [3]:
# Use the same HDF file and group name as when importing
test_fs = SXDMFrameset(file=file,
            dataset_name='test_data',
            scan_numbers=[294, 294, 294], # Doesnt work with a single value
            fill_num=3,
            restart_zoneplate=False,
            median_blur_algorithm='selective',
            )

Using --selective-- median_blur

Saved Scans: [294 294 294]
Current User Input: [294 294 294]
Importing Identical Scans. Reloading Saved Data...

Diameter Of The Zone Plate Is Set To b'150' microns
Outermost Zone Plate d Spacing Is Set To b'20' nanometers
The Size Of Your Detector Pixels Is Set To b'15' microns
The Detector Theta Value Is 16.7 Degrees and the Kev is 9 Kev

Hybrid Scan Shapes Are Equivalent
Pixel X Dimension: 30.96155019906851nm
Pixel Y Dimension: 31.242601799242426nm
All X and All Y Resolutions are within 0.5 nanometers from each other respectivley
X And Y Resolutions Are Within 0.5 nm Of Each Other


# Alignment

In [11]:
test_fs.alignment(reset=False)

Would You Like To Align On fluor Or roi?  fluor
Which - fluor - Would You Like To Center To/Return: ['Fe1' 'Fe2' 'Fe3' 'Fe4'] Fe3


# Centroid Analysis

In [12]:
test_fs.centroid_analysis(rows=test_fs.frame_shape()[1],
                            columns=test_fs.frame_shape()[2],
                            med_blur_distance=9,
                            med_blur_height=10,
                            stdev_min=25,
                            bkg_multiplier=0)

Which Scan Do You Want To Center Around? - Input Index -  0


Progress: 100%|██████████| 918/918 [00:08<00:00, 106.88pixles/s] 


Results Stored As self.results


In [13]:
test_fs.centroid_viewer()

 /Users/will/Documents/sxdm/sxdm/SXDM.py:393: UserWarning:The Starting Parameters In The Viewer May Not Be
Identical To The Parameters Used For The Analysis

A Centering Index of - 0 - Has Been Found. Would You Like To Use This Index To Center? y/n  y
Which - fluor - Would You Like To Center To/Return: ['Fe1' 'Fe2' 'Fe3' 'Fe4'] Fe3


Loading 2D: 100%|██████████| 26/26 [00:03<00:00,  7.23it/s]


SystemExit: 0

 /Users/will/anaconda3/envs/sxdm_pip/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning:To exit: use 'exit', 'quit', or Ctrl-D.

# ROI Segmentation

In [4]:
test_fs.roi_segmentation(bkg_multiplier=0, restart=False)

'SXDMFrameset' object has no attribute 'image_array' Initializing Image Array


A Centering Index of - 0 - Has Been Found. Would You Like To Use This Index To Center? y/n  y


Loading 2D: 100%|██████████| 33/33 [00:02<00:00, 12.28it/s]


In [5]:
test_fs.region_of_interest(rows=test_fs.frame_shape()[1],
                            columns=test_fs.frame_shape()[2],
                            med_blur_distance=9,
                            med_blur_height=100,
                            bkg_multiplier=0,
                            diff_segmentation=True,
                            slow=False)

Progress: 100%|██████████| 918/918 [00:14<00:00, 62.60pixles/s]  


Results Stored As self.roi_results


In [6]:
test_fs.roi_viewer()

In [7]:
def create_bounding_box_rois(fs):
    """Returns the ROI data for the rocking curve
    """
    main_roi = create_rois(fs)
    return main_roi[1]

def create_total_linescan(fs):
    """Obtains the summed data for each ROI generated.
    """
    main_roi = create_rois(fs)
    linescan_store = []
    for roi1 in main_roi[0]:
        linescan_store.append(np.nansum(roi1))
    return linescan_store

In [8]:
plt.plot(create_total_linescan(test_fs))